In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import time
import unittest
import math
import gym
import numpy as np

#env = gym.make('CartPole-v0')


env = gym.make('CartPole-v0')
env.reset()


# Qlearner for CartPole
class QLearner():
    def __init__(self, env):
        self.env = env
        self.epsilon_start = 1. # 1-epsilon probability of selecting best action. The epsilon is the greedy
        self.epsilon_end = 0.01 # final epsilon value
        self.epsilon_decay = 500 # steps before epsilon reaches epsilon_end
        self.epsilon_step = 0 #step tracker
        self.alpha = 0.1
        self.gamma = 0.99
        
        # get initial state, divide continuous states into discrete bins
#         self.bins = [np.linspace(env.observation_space.low[i], env.observation_space.high[i], 7) for i in range(4)]
        self.bins = []
        #self.bins.append(np.linspace(-2.4, 2.4, 5))
        #self.bins.append(np.linspace(-0.5, 0.5, 5))
        self.bins.append(np.linspace(-2.4, 2.4, 1))
        self.bins.append(np.linspace(-0.5, 0.5, 1))
        self.bins.append(np.linspace(-41.8, 41.8, 5))
        self.bins.append(np.linspace(-math.radians(50), math.radians(50), 5))
        self.reset()
        
        #initialize Q
#         self.Q = np.zeros((20, 20, 20, 20, 2))
        self.Q = {}
#         print(self.bins)

        self.episode_durations = []
        
    
    def getQ(self, s, action=None):
#         return self.Q[s[0], s[1], s[2], s[3], action]
#         print('getQ:', s, action)
        if not s in self.Q:
            self.Q[s] = [0, 0]

        if action is not None:
            return self.Q[s][action]
        else:
            return self.Q[s]
    
    
    def setQ(self, s, s_plus1, action, r):
#         print("Setting Q: ")
#         print("s: ", s, "s_plus1: ", s_plus1, " action: ", action, " r: ", r)
#         print("s:", s[0], s[1], s[2], s[3], "action: ", action)

        q = self.getQ(s, action)
    
        print "old q is ", q 
        
        
        
        max_q_splus1 = np.max(self.getQ(s_plus1))
        
        print "max q is ", max_q_splus1 
        
        print "R is ", r 
        
#         print("q: ", q, " max_q_splus1: ", max_q_splus1)
        new_q = q + self.alpha * (r + self.gamma * max_q_splus1 - q)
    
        print "new_q is:", new_q

        self.Q[s][action] = new_q
        
    def printQ(self):
        for s in self.Q:
            print(s, self.Q[s])
    
    def get_best_action(self, s):
        return np.argmax(self.getQ(s))
    
    def get_epsilon(self):
        return max(self.epsilon_end, self.epsilon_start + 
                      self.epsilon_step / self.epsilon_decay * (self.epsilon_end - self.epsilon_start))
    
    def get_training_action(self):
        epsilon = self.get_epsilon()

        if np.random.rand() >= epsilon:
            action = self.get_best_action(self.s)
        else:
            action = self.env.action_space.sample()
        
        self.epsilon_step += 1
        return action
    
    def reset(self):
        s = env.reset()
        self.s = tuple([np.asscalar(np.digitize(s[i], self.bins[i])) for i in range(4)])
    
    def step(self, action, training=False, step=0):
        observation, r, done, _ = self.env.step(action)
#         print("s0:", self.s)
#         print("s1:", s_plus1)
#         print("velocity:", s_plus1[3])
        s_plus1 = tuple([np.asscalar(np.digitize(observation[i], self.bins[i])) for i in range(4)])
        
        if training and done and step < 200:
            r = -100
        
        self.setQ(self.s, s_plus1, action, r)
            
        self.s = s_plus1
        
        return r, done
    
    def train(self, nb_episodes=2):
        
        for episode in range(nb_episodes):
            self.reset()
            episode_reward = 0
            
            while True:
                action = self.get_training_action()
                r, done = self.step(action, training=True, step=episode_reward+1)
                                
                episode_reward += r
                
                if done:
                    break;
            self.episode_durations.append(episode_reward + 1)
#             self.plot_durations()            
#             print("Episode %d reward: %d epsilon: %.4f" % (episode, episode_reward, self.get_epsilon()))

    def run(self):
        self.reset()
        self.env.render()
        episode_reward = 0
        while True:
            self.env.render()
            action = self.get_best_action(self.s)
            r, done = self.step(action)
            episode_reward += r
#             time.sleep(0.1)
            if done:
                self.env.close()
                break;
        
        self.env.render(close=True)
        print("Total Reward: ", episode_reward)
        
class TestQlearner(unittest.TestCase):
    def setUp(self):
        env = gym.make('CartPole-v0')
        self.qlearner = QLearner(env)
        self.qlearner.alpha = 0.01
        self.qlearner.gamma = 0.99

    def test_setQ(self):
        self.qlearner.setQ((4, 4, 4, 4), (4, 4, 4, 5), 1, 1)
        print(self.qlearner.Q)
        self.assertEqual(self.qlearner.getQ((4, 4, 4, 4), 1), 0.01)
        
        print(' ')
        self.qlearner.setQ((4, 4, 4, 5), (4, 4, 4, 4), 0, 1)
        print(self.qlearner.Q)
        newQ = self.qlearner.getQ((4, 4, 4, 5), 0)
        self.assertEqual(newQ, 0.010099)
        
        
    def runTest(self):
        self.test_setQ()
        

tester = TestQlearner()
tester.debug()

[2018-02-03 13:43:39,509] Making new env: CartPole-v0
[2018-02-03 13:43:39,539] Making new env: CartPole-v0


old q is  0
max q is  0
R is  1
new_q is: 0.01
{(4, 4, 4, 5): [0, 0], (4, 4, 4, 4): [0, 0.01]}
 
old q is  0
max q is  0.01
R is  1
new_q is: 0.010099
{(4, 4, 4, 5): [0.010099, 0], (4, 4, 4, 4): [0, 0.01]}


In [ ]:
env = gym.make('CartPole-v0')
qlearner = QLearner(env)

qlearner.train(nb_episodes=200)
qlearner.printQ()
plt.figure(2)
plt.clf()
plt.title('Training...')
plt.xlabel('Episode')
plt.ylabel('Duration')
plt.plot(qlearner.episode_durations)

[2018-02-03 13:43:44,421] Making new env: CartPole-v0


old q is  0
max q is  0
R is  1.0
new_q is: 0.1
old q is  0
max q is  0.1
R is  1.0
new_q is: 0.1099
old q is  0
max q is  0.1
R is  1.0
new_q is: 0.1099
old q is  0.1
max q is  0.1099
R is  1.0
new_q is: 0.2008801
old q is  0.1099
max q is  0.2008801
R is  1.0
new_q is: 0.2187971299
old q is  0.2187971299
max q is  0
R is  1.0
new_q is: 0.29691741691
old q is  0
max q is  0
R is  1.0
new_q is: 0.1
old q is  0
max q is  0
R is  1.0
new_q is: 0.1
old q is  0.1
max q is  0.1
R is  1.0
new_q is: 0.1999
old q is  0.1999
max q is  0.1999
R is  1.0
new_q is: 0.2997001
old q is  0.2997001
max q is  0.2997001
R is  1.0
new_q is: 0.3994003999
old q is  0
max q is  0.3994003999
R is  1.0
new_q is: 0.13954063959
old q is  0.13954063959
max q is  0.3994003999
R is  1.0
new_q is: 0.265127215221
old q is  0.3994003999
max q is  0.3994003999
R is  -100
new_q is: -9.6009990005
old q is  0
max q is  0
R is  1.0
new_q is: 0.1
old q is  0
max q is  0
R is  1.0
new_q is: 0.1
old q is  0
max q is  0
R is  

new_q is: 1.33142966034
old q is  1.42125390222
max q is  1.60164376903
R is  1.0
new_q is: 1.53769124513
old q is  1.60164376903
max q is  1.60164376903
R is  1.0
new_q is: 1.70004212526
old q is  1.70004212526
max q is  1.61985141754
R is  1.0
new_q is: 1.79040320307
old q is  0.864033801686
max q is  0.334053302355
R is  1.0
new_q is: 0.910701698451
old q is  0.19
max q is  1.14990945008
R is  1.0
new_q is: 0.384841035558
old q is  1.14990945008
max q is  1.14990945008
R is  1.0
new_q is: 1.24875954063
old q is  1.24875954063
max q is  1.19641830557
R is  1.0
new_q is: 1.34232899882
old q is  1.13837654176
max q is  1.19641830557
R is  1.0
new_q is: 1.24298429984
old q is  1.19641830557
max q is  1.24298429984
R is  1.0
new_q is: 1.2998319207
old q is  1.2998319207
max q is  1.34232899882
R is  1.0
new_q is: 1.40273929951
old q is  -3.68801470664
max q is  -17.4609864557
R is  1.0
new_q is: -4.94785089509
old q is  -24.7074696178
max q is  1.34232899882
R is  1.0
new_q is: -22.00383

old q is  3.31650388579
max q is  3.31650388579
R is  1.0
new_q is: 3.41318738191
old q is  2.54217004691
max q is  3.41318738191
R is  1.0
new_q is: 2.72585859303
old q is  2.72585859303
max q is  2.48373159457
R is  1.0
new_q is: 2.79916216159
old q is  2.48373159457
max q is  3.41318738191
R is  1.0
new_q is: 2.67326398592
old q is  3.41318738191
max q is  3.41318738191
R is  1.0
new_q is: 3.50977419453
old q is  3.50977419453
max q is  3.01478916433
R is  1.0
new_q is: 3.55726090234
old q is  3.01478916433
max q is  3.55726090234
R is  1.0
new_q is: 3.16547907723
old q is  3.55726090234
max q is  3.16547907723
R is  1.0
new_q is: 3.61491724075
old q is  3.16547907723
max q is  3.61491724075
R is  1.0
new_q is: 3.30680797634
old q is  3.61491724075
max q is  3.30680797634
R is  1.0
new_q is: 3.68079950634
old q is  3.30680797634
max q is  3.68079950634
R is  1.0
new_q is: 3.44052632983
old q is  3.68079950634
max q is  3.44052632983
R is  1.0
new_q is: 3.75333166236
old q is  2.1239

max q is  5.58771786774
R is  1.0
new_q is: 5.44844191998
old q is  5.58771786774
max q is  5.58771786774
R is  1.0
new_q is: 5.68213014987
old q is  5.68213014987
max q is  5.44844191998
R is  1.0
new_q is: 5.75331288497
old q is  5.44844191998
max q is  5.75331288497
R is  1.0
new_q is: 5.5731757036
old q is  5.75331288497
max q is  5.5731757036
R is  1.0
new_q is: 5.82972599113
old q is  5.5731757036
max q is  5.82972599113
R is  1.0
new_q is: 5.69300100636
old q is  5.82972599113
max q is  5.82972599113
R is  1.0
new_q is: 5.92389626513
old q is  5.92389626513
max q is  5.69300100636
R is  1.0
new_q is: 5.99511373825
old q is  5.69300100636
max q is  5.99511373825
R is  1.0
new_q is: 5.81721716581
old q is  5.99511373825
max q is  5.81721716581
R is  1.0
new_q is: 6.07150686384
old q is  5.81721716581
max q is  6.07150686384
R is  1.0
new_q is: 5.93657462875
old q is  6.07150686384
max q is  6.07150686384
R is  1.0
new_q is: 6.16543535698
old q is  6.16543535698
max q is  5.9365746

...

old q is  68.019796511
max q is  68.0555810146
R is  1.0
new_q is: 68.0553193803
old q is  68.0555810146
max q is  68.0553193803
R is  1.0
new_q is: 68.0874995318
old q is  68.0553193803
max q is  68.0874995318
R is  1.0
new_q is: 68.090449896
old q is  68.0874995318
max q is  68.090449896
R is  1.0
new_q is: 68.1197041183
old q is  68.090449896
max q is  68.1197041183
R is  1.0
new_q is: 68.1252556141
old q is  68.1197041183
max q is  68.1252556141
R is  1.0
new_q is: 68.1521340123
old q is  68.1252556141
max q is  68.1521340123
R is  1.0
new_q is: 68.1597913199
old q is  68.1521340123
max q is  68.1521340123
R is  1.0
new_q is: 68.1839818783
old q is  68.1839818783
max q is  68.1597913199
R is  1.0
new_q is: 68.2134030311
old q is  68.1597913199
max q is  68.2134030311
R is  1.0
new_q is: 68.196939088
old q is  68.2134030311
max q is  68.196939088
R is  1.0
new_q is: 68.2435596977
old q is  68.196939088
max q is  68.2435596977
R is  1.0
new_q is: 68.2333575893
old q is  68.2435596977

new_q is: 54.998998456
old q is  54.9736192992
max q is  54.998998456
R is  1.0
new_q is: 55.0211582164
old q is  54.998998456
max q is  55.0211582164
R is  1.0
new_q is: 55.0461932738
old q is  55.0211582164
max q is  55.0461932738
R is  1.0
new_q is: 55.0686155289
old q is  55.0461932738
max q is  55.0686155289
R is  1.0
new_q is: 55.0933668838
old q is  55.0686155289
max q is  55.0933668838
R is  1.0
new_q is: 55.1159972975
old q is  55.0933668838
max q is  55.1159972975
R is  1.0
new_q is: 55.1405139279
old q is  55.1159972975
max q is  55.1405139279
R is  1.0
new_q is: 55.1633084466
old q is  55.1405139279
max q is  55.1405139279
R is  1.0
new_q is: 55.1853734139
old q is  55.1853734139
max q is  20.1995026946
R is  1.0
new_q is: 51.7665868393
old q is  20.1995026946
max q is  24.9989928593
R is  1.0
new_q is: 20.7544527182
old q is  24.9989928593
max q is  20.7544527182
R is  1.0
new_q is: 24.6537843924
old q is  20.7544527182
max q is  24.6537843924
R is  1.0
new_q is: 21.219732

max q is  69.1324103372
R is  1.0
new_q is: 69.1788916476
old q is  69.1324103372
max q is  69.1788916476
R is  1.0
new_q is: 69.1678795766
old q is  69.1788916476
max q is  69.1678795766
R is  1.0
new_q is: 69.2086225609
old q is  69.1678795766
max q is  69.2086225609
R is  1.0
new_q is: 69.2027452524
old q is  69.2086225609
max q is  69.2027452524
R is  1.0
new_q is: 69.2388320848
old q is  69.2027452524
max q is  69.2388320848
R is  1.0
new_q is: 69.2371151036
old q is  69.2388320848
max q is  69.2371151036
R is  1.0
new_q is: 69.2694232716
old q is  69.2371151036
max q is  69.2694232716
R is  1.0
new_q is: 69.2710764971
old q is  69.2694232716
max q is  69.2710764971
R is  1.0
new_q is: 69.3003175177
old q is  69.2710764971
max q is  69.3003175177
R is  1.0
new_q is: 69.3047002817
old q is  69.3003175177
max q is  69.3047002817
R is  1.0
new_q is: 69.3314510938
old q is  69.3047002817
max q is  69.3314510938
R is  1.0
new_q is: 69.3380439118
old q is  69.3314510938
max q is  69.331

R is  1.0
new_q is: 71.1120265602
old q is  71.0727862492
max q is  71.1120265602
R is  1.0
new_q is: 71.1055982538
old q is  71.1120265602
max q is  71.1055982538
R is  1.0
new_q is: 71.1402781313
old q is  71.1055982538
max q is  71.1402781313
R is  1.0
new_q is: 71.1379259634
old q is  71.1402781313
max q is  71.1379259634
R is  1.0
new_q is: 71.1689049885
old q is  71.1379259634
max q is  71.1689049885
R is  1.0
new_q is: 71.1698549609
old q is  71.1689049885
max q is  71.1698549609
R is  1.0
new_q is: 71.1978301308
old q is  71.1698549609
max q is  71.1978301308
R is  1.0
new_q is: 71.2014546478
old q is  71.1978301308
max q is  71.1978301308
R is  1.0
new_q is: 71.2266323007
old q is  71.2266323007
max q is  71.2014546478
R is  1.0
new_q is: 71.2529130808
old q is  71.2014546478
max q is  71.2529130808
R is  1.0
new_q is: 71.235347578
old q is  71.2529130808
max q is  71.235347578
R is  1.0
new_q is: 71.2799211829
old q is  71.235347578
max q is  71.2799211829
R is  1.0
new_q is:

R is  1.0
new_q is: 67.3996903975
old q is  67.3823588355
max q is  67.3996903975
R is  1.0
new_q is: 67.4166923013
old q is  67.3996903975
max q is  67.4166923013
R is  1.0
new_q is: 67.4339738956
old q is  67.4166923013
max q is  67.4166923013
R is  1.0
new_q is: 67.449275609
old q is  67.449275609
max q is  67.4339738956
R is  1.0
new_q is: 67.4803114637
old q is  67.4339738956
max q is  67.4803114637
R is  1.0
new_q is: 67.4711273409
old q is  67.4803114637
max q is  67.4711273409
R is  1.0
new_q is: 67.5119219241
old q is  67.4711273409
max q is  67.5119219241
R is  1.0
new_q is: 67.5076948773
old q is  67.5119219241
max q is  67.5076948773
R is  1.0
new_q is: 67.5439915245
old q is  67.5076948773
max q is  67.5439915245
R is  1.0
new_q is: 67.5437805505
old q is  67.5439915245
max q is  67.5437805505
R is  1.0
new_q is: 67.5764266466
old q is  67.5437805505
max q is  67.5764266466
R is  1.0
new_q is: 67.5794687335
old q is  67.5764266466
max q is  67.5794687335
R is  1.0
new_q is

In [7]:
qlearner.run()

old q is  83.80023171
max q is  82.6958560084
R is  1.0
new_q is: 83.7070982838
old q is  82.6958560084
max q is  83.7070982838
R is  1.0
new_q is: 82.8132731377
old q is  83.7070982838
max q is  82.8132731377
R is  1.0
new_q is: 83.6349024961
old q is  82.8132731377
max q is  83.6349024961
R is  1.0
new_q is: 82.911801171
old q is  83.6349024961
max q is  82.911801171
R is  1.0
new_q is: 83.5796805624
old q is  82.911801171
max q is  83.5796805624
R is  1.0
new_q is: 82.9950094296
old q is  83.5796805624
max q is  82.9950094296
R is  1.0
new_q is: 83.5382184397
old q is  82.9950094296
max q is  83.5382184397
R is  1.0
new_q is: 83.0657921122
old q is  83.5382184397
max q is  83.0657921122
R is  1.0
new_q is: 83.5079100148
old q is  83.0657921122
max q is  83.5079100148
R is  1.0
new_q is: 83.1264959924
old q is  83.5079100148
max q is  83.1264959924
R is  1.0
new_q is: 83.4866421166
old q is  83.1264959924
max q is  83.4866421166
R is  1.0
new_q is: 83.1790239627
old q is  83.48664211

In [2]:
learner = QLearner(env)
learner.train()
learner.run()

('Total Reward: ', 14.0)
